<a href="https://colab.research.google.com/github/jinseriouspark/polaris-llm/blob/main/gemma_finetuning_model_convert_gguf_and_upload_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# gemma_finetuning_model_convert_gguf_and_upload_hf
- ref1 : https://github.com/ggerganov/llama.cpp/discussions/2948
- ref2 : https://github.com/ggerganov/llama.cpp/issues/7897


1. download model snapshot
2. convert gguf
    
    - you need to remove the check part in `/content/llama.cpp/gguf-py/gguf/gguf_writer.py`
    ```
    if key in self.kv_data:
        raise ValueError(f'Duplicated key name {key!r}')```
3. upload your gguf file on hugging face


In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import snapshot_download
model_id="seriouspark/gemma-7b-it-persona"
snapshot_download(repo_id=model_id, local_dir="gemma-7b-it-persona",
                  local_dir_use_symlinks=False, revision="main")

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git

In [ ]:
!pip install -r ./llama.cpp/requirements.txt

In [ ]:
# before run this cell,
# you need to remove the check part in
#    - /content/llama.cpp/gguf-py/gguf/gguf_writer.py
##  if key in self.kv_data:
##             raise ValueError(f'Duplicated key name {key!r}')

!python ./llama.cpp/convert-hf-to-gguf.py -h

In [ ]:
!python ./llama.cpp/convert-hf-to-gguf.py gemma-7b-it-persona \
  --outfile gemma-7b-it-persona.gguf \
  --outtype q8_0

In [ ]:
ls -lash gemma-7b-it-persona.gguf

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

model_id = "seriouspark/gemma-7b-it-persona-gguf"
api.create_repo(model_id, exist_ok=True, repo_type="model")
api.upload_file(
    path_or_fileobj="gemma-7b-it-persona.gguf",
    path_in_repo="gemma-7b-it-persona.gguf",
    repo_id=model_id,
)